In [10]:
import pandas as pd
pd.options.display.max_columns = None


In [11]:
songs = pd.read_csv('data/spotify_data.csv', index_col=[0])
# before dropping duplicates
# len(songs[songs['artists'] == 'Jack Harlow'])


In [27]:
songs = songs.drop_duplicates(['track_name','artists'])
# after dropping duplicates
len(songs[songs['artists'] == 'Jack Harlow'])
songs.dropna(inplace = True)
len(songs)

X = songs.drop(columns = ["track_id", "artists", "album_name", "track_name", "track_genre"])
y = songs["track_genre"]




In [30]:
y

0            acoustic
1            acoustic
2            acoustic
3            acoustic
4            acoustic
             ...     
113995    world-music
113996    world-music
113997    world-music
113998    world-music
113999    world-music
Name: track_genre, Length: 81343, dtype: object

In [31]:
from sklearn.preprocessing import LabelEncoder
# numerically encode Label 
le = LabelEncoder()
y = le.fit_transform(y)

In [54]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay, accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle=True)

model = DecisionTreeClassifier()
model.fit(X_train, y_train)
predictions = model.predict(X_test)
score = accuracy_score(y_test, predictions)

score

0.24832503534329092

In [59]:
# ytrue = pd.Series(np.argmax(y_test, axis= 1), name="actual")
# ypred = pd.Series(np.argmax(predictions, axis= 1), name="predicted")

pd.crosstab(y_test, predictions)

col_0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112
row_0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,22,2,1,1,3,2,0,5,3,1,0,6,11,0,7,5,0,1,1,4,1,1,1,1,0,1,1,0,0,1,1,2,0,1,4,0,2,0,1,1,0,1,0,2,1,4,0,0,2,0,2,1,1,1,0,3,0,0,1,0,0,2,2,2,2,2,1,0,0,1,8,1,1,0,2,1,1,0,0,4,1,1,1,0,0,0,1,0,0,0,0,3,3,1,4,2,0,0,3,2,1,1,0,2,0,2,1,0,0,0,4,2,4
1,2,30,0,0,0,1,0,10,1,2,5,0,2,4,0,0,0,9,2,0,0,3,0,1,2,2,1,2,0,0,1,0,1,0,1,0,1,1,2,1,0,1,0,0,1,5,0,0,0,1,0,0,0,1,3,0,0,0,5,0,9,3,1,1,0,0,3,0,1,5,1,0,0,2,1,1,0,0,0,0,0,1,7,0,0,0,0,0,0,0,0,0,4,0,0,2,0,0,3,1,0,0,0,1,1,0,3,7,1,0,13,1,1
2,1,0,11,5,1,6,1,0,2,4,0,1,1,1,0,1,0,1,0,7,1,2,0,2,0,5,0,0,0,1,5,2,2,1,1,1,1,1,1,0,0,2,0,0,4,0,0,3,2,1,1,1,0,1,0,1,1,3,1,4,0,1,3,0,2,8,0,5,3,1,2,0,2,0,3,0,0,1,1,6,1,1,1,2,1,0,2,1,0,0,2,1,1,2,1,2,0,1,0,1,0,0,0,1,0,3,3,0,1,1,0,0,2
3,0,0,3,2,0,2,0,0,1,1,0,0,0,0,0,0,0,0,0,0,3,0,0,2,0,3,0,0,0,0,1,2,0,3,0,0,1,0,0,1,0,0,0,0,0,0,0,2,1,0,0,1,0,1,1,0,0,3,0,2,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,2,0,0,2,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1
4,4,1,0,0,34,8,0,3,2,1,2,10,1,0,2,4,6,1,1,2,0,0,0,0,0,0,2,0,0,0,0,1,2,0,2,0,2,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,3,4,0,0,1,1,0,0,2,1,0,1,0,0,0,0,0,0,0,2,0,8,3,0,0,18,0,2,0,0,2,0,0,0,0,0,0,0,0,0,1,0,0,0,2,1,0,3,1,1,2,0,0,0,0,1,1,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,0,2,0,0,0,0,0,0,2,0,2,0,0,0,0,0,0,0,0,0,0,0,0,7,1,0,0,0,1,1,1,0,2,0,0,0,0,0,2,0,0,0,0,0,0,0,0,1,1,4,0,0,0,1,0,2,0,1,2,0,0,2,0,0,0,0,0,0,0,3,1,0,1,21,0,0,0,0,0,0,0,0,2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,1,10,5,0,0,0
109,1,0,1,0,1,3,3,0,0,0,3,0,0,0,1,1,0,8,0,0,0,0,3,4,0,4,0,5,2,2,1,0,5,2,0,0,1,0,2,1,0,2,0,1,2,0,1,1,0,2,2,0,0,2,2,0,0,0,5,1,0,0,1,0,0,0,0,0,0,0,0,0,0,7,1,1,0,0,0,0,1,1,0,12,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,2,0,0,0,3,44,3,1,1
110,2,15,3,0,2,2,3,2,1,0,2,1,3,5,0,3,0,5,0,0,0,1,0,0,0,1,3,4,0,1,1,0,7,1,1,0,1,0,0,0,0,5,0,2,0,0,2,1,2,0,1,0,0,0,5,2,0,0,3,0,1,2,1,0,0,3,0,0,1,4,4,0,0,9,3,3,0,0,0,0,0,1,1,1,4,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,2,1,0,2,1,0,20,0,0
